In [1]:
import os
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt; 
 
# Importing sklearn libraries
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
 
# Importing hypopt library for grid search
#from hypopt import GridSearch
 
# Importing Keras libraries
from keras.utils import np_utils
from keras.models import Sequential
from keras.applications import VGG16, MobileNet, ResNet50
from keras_contrib.applications import ResNet18, ResNet34

from keras.applications import imagenet_utils
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import Dense, Conv2D, MaxPooling2D,Input
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D
from keras.models import load_model
import keras.backend as K
from keras.models import Model
from keras.layers import multiply, LocallyConnected2D, Lambda, ActivityRegularization

import cv2

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="5"

print("[INFO] loading network...")
 
# chop the top dense layers, include_top=False
#model = VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=(110, 110, 3)))
#model = MobileNet(weights="imagenet", include_top=False,input_tensor=Input(shape=(110, 110, 3)))
model = ResNet50(weights="imagenet", include_top=False,input_tensor=Input(shape=(110, 110, 3)))
#model.layers.pop()
model = Model(inputs=model.inputs, outputs=model.layers[-33].output)
model.summary()

[INFO] loading network...
Instructions for updating:
Colocations handled automatically by placer.


/home/xxing/anaconda3/envs/myenv3/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 110, 110, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 116, 116, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 55, 55, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 55, 55, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [3]:
#data input:
X=np.load('/data/scratch/xxing/adni_dl/CN_AD_Dynamic_ADNI2_MRI_data_3d.npy')
X=X/255.0
y=np.load('/data/scratch/xxing/adni_dl/CN_AD_Dynamic_ADNI2_MRI_label.npy')


In [4]:
bs=8
num_classes=2

 
X_features = model.predict(X, batch_size=32)
print(X_features.shape)

#Y=np_utils.to_categorical(y, num_classes)
"""
from sklearn.model_selection import train_test_split
train_features, test_features, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=32) 


# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, num_classes)
Y_test = np_utils.to_categorical(y_test, num_classes)
"""

(99, 7, 7, 1024)


'\nfrom sklearn.model_selection import train_test_split\ntrain_features, test_features, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=32) \n\n\n# convert class vectors to binary class matrices\nY_train = np_utils.to_categorical(y_train, num_classes)\nY_test = np_utils.to_categorical(y_test, num_classes)\n'

In [5]:
#K-fold cross validation
from sklearn.model_selection import StratifiedKFold
seed=9
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

In [7]:
import gc
cvscores = []
i=0
for train, test in kfold.split(X_features, y):
    #classifier
    
    
    #model_transfer.add(GlobalAveragePooling2D(input_shape=X_features.shape[1:]))
    in_lay = Input(X_features.shape[1:])
    attn_layer = Conv2D(512, kernel_size = (1,1), padding = 'same', activation = 'relu')(in_lay)
    attn_layer = Conv2D(128, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
    attn_layer = Conv2D(64, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
    attn_layer = Conv2D(1, 
                    kernel_size = (1,1), 
                    padding = 'valid', 
                    activation = 'sigmoid')(attn_layer)
    attn_layer = ActivityRegularization(l1 = 1e-2, name = 'Attention_Focus_Penalty')(attn_layer)
    pt_depth=1024

    # fan it out to all of the channels
    up_c2_w = np.ones((1, 1, 1, pt_depth))
    up_c2 = Conv2D(pt_depth, kernel_size = (1,1), padding = 'same', 
               activation = 'linear', use_bias = False, weights = [up_c2_w])
    up_c2.trainable = False
    attn_layer = up_c2(attn_layer)

    mask_features = multiply([attn_layer, in_lay])
    gap_features = GlobalAveragePooling2D()(mask_features)
    gap_mask = GlobalAveragePooling2D()(attn_layer)
    # to account for missing values from the attention model
    gap = Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features, gap_mask])
    gap_dr = Dropout(0.5)(gap)
    dr_steps = Dropout(0.25)(Dense(512, activation = 'elu')(gap_dr))
    out_layer = Dense(1, activation = 'sigmoid')(dr_steps) # linear is what 16bit did
    model_transfer= Model(inputs = [in_lay], outputs = [out_layer])
    #model_transfer.summary()
    opt = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=True)
    model_transfer.compile(loss='binary_crossentropy', optimizer=opt,
              metrics=['accuracy'])
    # Creating a checkpointer
    checkpointer = ModelCheckpoint(filepath='TFL_Sampled_Dynamic_AV45_CNvsAD_.best_'+str(i)+'.hdf5',
                               verbose=0,save_best_only=True,monitor='val_acc', mode='max')

    history = model_transfer.fit(X_features[train], y[train], batch_size=32, epochs=200,
          validation_data=(X_features[test], y[test]), callbacks=[checkpointer],
          verbose=0, shuffle=True)
    # evaluate the model
    modelpath='TFL_Sampled_Dynamic_AV45_CNvsAD_.best_'+str(i)+'.hdf5'
    model_evaluate = load_model(modelpath)

    scores = model_evaluate.evaluate(X_features[test], y[test], verbose=0)
    print("%s: %.2f%%" % (model_evaluate.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    i=i+1
    
    K.clear_session()
    gc.collect()
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
acc: 85.00%
acc: 80.00%
acc: 80.00%
acc: 85.00%
acc: 78.95%
81.79% (+/- 2.65%)


In [7]:
"""
# Creating a checkpointer
checkpointer = ModelCheckpoint(filepath='test.best.hdf5',
                               verbose=1,save_best_only=True,monitor='val_acc', mode='max')

history = model_transfer.fit(train_features, Y_train, batch_size=bs, epochs=100,
          validation_data=(test_features, Y_test), callbacks=[checkpointer],
          verbose=0, shuffle=True)
"""

"\n# Creating a checkpointer\ncheckpointer = ModelCheckpoint(filepath='test.best.hdf5',\n                               verbose=1,save_best_only=True,monitor='val_acc', mode='max')\n\nhistory = model_transfer.fit(train_features, Y_train, batch_size=bs, epochs=100,\n          validation_data=(test_features, Y_test), callbacks=[checkpointer],\n          verbose=0, shuffle=True)\n"

In [8]:
"""
import matplotlib.pyplot as plt
print(features.shape)
print(features[0,:,:,0])
plt.imshow(features[0,:,:,120])
"""

'\nimport matplotlib.pyplot as plt\nprint(features.shape)\nprint(features[0,:,:,0])\nplt.imshow(features[0,:,:,120])\n'